In [95]:
import os
import re

import pandas as pd

import nltk

import email

In [3]:
os.getcwd()

'/Users/devnambi/Documents/GitHub/pruitt-emails'

In [70]:
files = [f for f in os.listdir('txt') if f.endswith('.txt')]

In [71]:
files

['Produce-1600-2541-Redacted.txt',
 'Produce-2000.txt',
 'Produce-6000.txt',
 'Produce-Box-5-Redacted.txt',
 'Produce-Box-5.txt',
 'Produce-Box-6.txt']

In [72]:
for f in files:
    txt_path = os.path.join('txt', f)
    print txt_path

txt/Produce-1600-2541-Redacted.txt
txt/Produce-2000.txt
txt/Produce-6000.txt
txt/Produce-Box-5-Redacted.txt
txt/Produce-Box-5.txt
txt/Produce-Box-6.txt


In [73]:
reader = open ('txt/Produce-1600-2541-Redacted.txt','r')
entire_file = reader.read()

In [ ]:
entire_file

In [77]:
emails_raw = re.split('From:', entire_file, flags=re.MULTILINE)

In [78]:
emails_df = pd.DataFrame(emails_raw)
emails_df.columns = ['email']
emails_df['email'] = 'From:'+emails_df['email']
emails_df['from'] = ''
emails_df['to'] = ''
emails_df['subject'] = ''
emails_df['date'] = ''
emails_df['bate_stamp'] = ''
emails_df['message'] = ''

In [79]:
# create a function called cleanup_email
def cleanup_email(x):
    return x.replace("\n", " ").decode('utf-8').encode("ascii",'ignore').replace("\x0c","")

In [80]:
# clean up the email formatting a bit
emails_df['email'] = emails_df['email'].apply(cleanup_email)

In [81]:
# create a function to find the bate stamps in each email thread
def find_batestamps(x):
    matches = re.findall('Feb-17-2017 \d{4,7}', x)
    batestamps = ''
    for match in matches:
        batestamps = batestamps + match + ' '
    return batestamps.rstrip()

In [82]:
# identify the bate stamps in each email
emails_df['bate_stamp'] = emails_df['email'].apply(find_batestamps)

In [111]:
# create a function to find the sender in each email thread
def find_sender(x):
    match = re.search('From:\s?(?P<sender>.(0,100))\s?(Sent:|To:|Date:|Subject).+', x, flags=re.IGNORECASE)
    sender = ''
    if match:
        sender = match.group('sender')
    else:
        match2 = re.search('From:\s?(?P<sender>.*)\s?(To:)', x, flags=re.IGNORECASE)
        if match2:
            sender = match2.group('sender')
    return sender.rstrip()

In [112]:
find_sender(emails_df.email[15])

'Lynne Ross [mailto:toswim2@yahoo.com] Sent: Thursday, April 04, 2013 5:00 PM'

In [105]:
emails_df['from'] = emails_df['email'].apply(find_sender)

['From']

In [107]:
emails_df.head(100)

,email,from,to,subject,date,bate_stamp,message
0,From:,,,,,,
1,From: To: Cc: Subject: Date: andrewpmiller@v...,,,,,Feb-17-2017 001635 Feb-17-2017 001636 Feb-17-2...,
2,From: To: Cc: Subject: Date: Charlie Brown '...,,,,,,
3,From: Lynne Ross [mailto:toswim2@yahoo.com] Se...,Lynne Ross [mailto:toswim2@yahoo.com],,,,Feb-17-2017 001638,
4,From: To: Cc: Subject: Date: Rufus Edmisten ...,,,,,,
5,From: Lynne Ross [mailto:toswim2@yahoo.com] Se...,Lynne Ross [mailto:toswim2@yahoo.com],,,,Feb-17-2017 001639 Feb-17-2017 001640,
6,From: To: Cc: Subject: Date: Drew Ketterer t...,,,,,,
7,From: Lynne Ross <toswim2@yahoo.com> To: Rober...,Lynne Ross <toswim2@yahoo.com> To: Robert Abra...,,,,Feb-17-2017 001641 Feb-17-2017 001642,
8,"From: To: Abrams, Robert Lynne Ross; Bruce Ba...",,,,,,
9,From: Lynne Ross [mailto:toswim2@yahoo.com] Se...,Lynne Ross [mailto:toswim2@yahoo.com],,,,Feb-17-2017 001643,


In [58]:
emails_df.email[15]

"From: Lynne Ross [mailto:toswim2@yahoo.com] Sent: Thursday, April 04, 2013 5:00 PM To: Robert Abrams; Bruce Babbitt; Bill Baxley; Richard Bryan; Pam & Mike Carter; Bob Stephan; Joe Curran; Frankie Sue Del Papa; Bob Del Tufo; Jim Hood; James E. Doyle; Drew Edmondson; Lee Fisher; Dave Frohnmayer; Jan Graham; JENNIFER GRANHOLM; Scott Harshbarger; Andrew Ketterer; Peter Harvey; Oliver Koppell; Peg Lautenschlager; Patrick Lynch; JD MacFarlane; Patsy Madrid; Janet Mills; Jeff Modisett; Mary Ann Myers; ed pittman; Dennis J. Roberts; Steve Rowe; Steve Six; Smith, Greg; Robert Spagnoletti; Mary Sue Sue Terry; Tony Troy; James E. Tierney; Paul Van Dam; John Van de Kamp; Mark White; Richard Opper; Gale A. Norton; Duane Woodard; Carl Danberg; Richard Gebelein; richard weir; Charlie Brown; John Easton; Bill McCollum; Andy Miller; Richard Doran; Jim Smith; Thurbert E. Baker; Karen McKay; Alicia Limtiaco; Mark J. Bennett; Rosenthal Steve; Steve Carter; Jeff Amestoy; Francis X Bellotti; jim shannon; 

In [63]:
find_batestamps(emails_df.email[1])

'Feb-17-2017 001635 Feb-17-2017 001636 Feb-17-2017 001637'

In [39]:
re.match('',emails_df.email[1])

"From:\nTo:\n\nCc:\nSubject:\nDate:\n\nandrewpmiller@verizon.net\ntoswim2@yahoo.com; rabrams@stroock.com; bruce.babbitt@raintreeventures.com; bbaxley@baxleydillard.com;\nrbryan@lionelsawyer.com; pamela.l.carter@cummins.com; xgeneral@aol.com; agjoe@comcast.net;\nrenofsdp@aol.com; rdeltufo@skadden.com; jimhoodag@hotmail.com; jdoyle@foley.com;\ndrew.edmondson@oag.ok.gov; LFisher@ceosforcities.org; dave.frohnmayer@harrang.com;\njan@grahamlawoffices.com; Granholm@jennifergranholm.com; scott.harshbarger@mercerdelta.com;\ndrewketterer@roadrunner.com; pcharvey@pbwt.com; okoppell@koppellaw.com; Lautenschlager@bauerbach.com; patrick@patricklynchgroup.com; jdmac55@comcast.net; patriciamadrid100@yahoo.com;\njanet.t.mills@gmail.com; jeff.modisett@snrdenton.com; whmam@comcast.net; pittman@pdmd.biz;\ndroberts@djrlaw.com; gsrowe@maine.rr.com; six@stuevesiegel.com; GREG.SMITH@MCLANE.com;\nRSpagnoletti@schertlerlaw.com; msterry@embarqmail.com; ttroy@eckertseamans.com; jtierney@clinic.net;\nvandamussenat